In [30]:
%matplotlib inline
import pandas as pd
import numpy as np
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))# nrows=1048576)

In [31]:
data.head()

,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06


In [50]:
columns=['CountryCode']
countries = pd.read_csv("data/Country.csv", usecols=(columns))

In [51]:
countries.head()

,CountryCode
0,AFG
1,ALB
2,DZA
3,ASM
4,ADO


In [52]:
# find some measures that seem important and are often present, filter countries with all those intact, train network

In [53]:
columns=['LongDefinition', 'SeriesCode']
series = pd.read_csv("data/Series.csv", usecols=(columns))

In [54]:
series.head()

,SeriesCode,LongDefinition
0,BN.KLT.DINV.CD,Foreign direct investment are the net inflows ...
1,BX.KLT.DINV.WD.GD.ZS,Foreign direct investment are the net inflows ...
2,BX.KLT.DINV.CD.WD,Foreign direct investment refers to direct inv...
3,BM.KLT.DINV.GD.ZS,Foreign direct investment are the net inflows ...
4,BN.TRF.KOGT.CD,Net capital account records acquisitions and d...


In [55]:
print (len(countries))
print (len(series))

247
1345


In [56]:
#df[(df.A == 1) & (df.D == 6)]
seriesCounts = data['IndicatorCode'].value_counts()

In [57]:
filteredSeries = seriesCounts['SP.POP.TOTL']

13484